In [1]:
import os
import sys
from quetzal.os.parallel_call import parallel_call_notebook, parallel_call_notebooks

In [2]:
if not os.path.exists('log/'):
    os.makedirs('log/')

# Launcher

This notebook automatically launches all operations for a complete model run. One can decide to skip network preparation steps, as they take multiple hours for the region of Germany and the networks are readily included in the repository.

Detailed explainations of certain steps can be found in the corresponding notebook. All ASSUMPTIONS are gathered in the `parameters.xls` file. This file also includes scenarios.

In [3]:
kwargs = {'workers':4, 'errout_suffix':True, 'sleep':1,'stdout_path':r'log/out.txt', 'stderr_path':r'log/err.txt'}

In [4]:
scenarios = ['base'] #['2035_base', 'CO2-tax_210', 'road_charge']

In [5]:
import time
start = time.time()

# Network preparation

Execute all `prep1` and `prep2` steps.

In [7]:
# Network generation
parallel_call_notebook('prep10_zones.ipynb', arg_list=scenarios, **kwargs)
parallel_call_notebooks(
    ('prep11_road.ipynb', scenarios),
    ('prep12_rail_coach.ipynb', scenarios),
    ('prep14_bus.ipynb', scenarios),
    **kwargs)
parallel_call_notebook('prep13_air.ipynb', arg_list=scenarios, **kwargs)

0 prep12_rail_coach.py base
1 prep14_bus.py base
subprocess **prep14_bus.py1 base** terminated with an error.
1763 seconds
0 prep13_air.py base
28 seconds


In [8]:
# Access/egress distances to PT services based on networks and census data
parallel_call_notebook('prep15_census_distances.ipynb', arg_list=scenarios, **kwargs)

0 prep15_census_distances.py base
7805 seconds


In [9]:
# Network aggregation
# First cluster stops, then aggregate trips
parallel_call_notebook('prep20_cluster.ipynb', arg_list=scenarios, **kwargs)
parallel_call_notebook('prep21_aggregate_pt.ipynb', arg_list=scenarios, **kwargs)
parallel_call_notebook('prep22_access_egress_road.ipynb', arg_list=scenarios, **kwargs)

0 prep20_cluster.py base
3099 seconds
0 prep21_aggregate_pt.py base
1220 seconds
0 prep22_access_egress_road.py base
98 seconds


# Pathfinders and level of service

In [5]:
# Road
parallel_call_notebook('prep30_pathfinder_road.ipynb', arg_list=scenarios, **kwargs)
parallel_call_notebook('prep31_prices_car.ipynb', arg_list=scenarios, **kwargs)

0 prep30_pathfinder_road.py base
266 seconds
0 prep31_prices_car.py base
72 seconds


In [5]:
# PT
parallel_call_notebook('prep30_pathfinder_pt.ipynb', arg_list=scenarios, **kwargs)
parallel_call_notebook('prep31_prices_pt.ipynb', arg_list=scenarios, **kwargs)
# terminates with error from 'assert manual' if there is a saved versions of footpaths
# so that it doesn't have to compute them again
parallel_call_notebook('prep33_non-motor.ipynb', arg_list=scenarios, **kwargs)

0 prep30_pathfinder_pt.py base
4586 seconds
0 prep31_prices_pt.py base
117 seconds
0 prep33_non-motor.py base
subprocess **prep33_non-motor.py0 base** terminated with an error.
47 seconds


# Estimation

In [6]:
# Calibration data preparation
# NOTE: only applicapble if you have access to the calibration dataset
parallel_call_notebook('cal10_input_data.ipynb', arg_list=scenarios, **kwargs)
#parallel_call_notebook('cal1x_validation_data.ipynb', arg_list=scenarios, **kwargs)
parallel_call_notebook('cal11_los_columns.ipynb', arg_list=scenarios, **kwargs)
#parallel_call_notebook('cal19_vot.ipynb', arg_list=scenarios, **kwargs)

0 cal10_input_data.py base
subprocess **cal10_input_data.py0 base** terminated with an error.
22 seconds
0 cal1x_validation_data.py base
subprocess **cal1x_validation_data.py0 base** terminated with an error.
4 seconds
0 cal11_los_columns.py base
72 seconds


In [7]:
# Estimation
parallel_call_notebook('cal20_estimation.ipynb', arg_list=scenarios, **kwargs)

0 cal20_estimation.py base
5475 seconds


# Modelling

Except for the logit model step, all modelling steps require access to the OD volumes dataset.

In [9]:
# Generation of the OD matrix
parallel_call_notebook('model_volumes.ipynb', arg_list=scenarios, **kwargs)

0 model_volumes.py 2035_base
subprocess **model_volumes.py0 2035_base** terminated with an error.
330 seconds


In [10]:
# Mode choice probabilities
parallel_call_notebook('model_logit.ipynb', arg_list=scenarios, **kwargs)

0 model_logit.py 2035_base
1019 seconds


In [11]:
# Assignment
parallel_call_notebook('model_assignment.ipynb', arg_list=scenarios, **kwargs)
#parallel_call_notebook('model_assignment_inner-zonal.ipynb', arg_list=scenarios, **kwargs)

0 model_assignment.py 2035_base
641 seconds


In [12]:
# Post-processing
parallel_call_notebook('model_emissions.ipynb', arg_list=scenarios, **kwargs)

0 model_emissions.py 2035_base
372 seconds


In [9]:
# Total execution time
int(time.time() - start)

2539